# 글자 수준의 번역기 만들기

## 참고

https://wikidocs.net/24996


## 데이터 출처

http://www.manythings.org/anki/  
fra-eng.zip의 fra.txt 파일을 사용

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# http = urllib3.PoolManager()
# url ='http://www.manythings.org/anki/fra-eng.zip'
# filename = 'fra-eng.zip'
# path = os.getcwd()
# zipfilename = os.path.join(path, filename)
# with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
#     shutil.copyfileobj(r, out_file)

# with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
#     zip_ref.extractall(path)

## Preprocessing

In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 194513


In [4]:
lines # 영어 - 프랑스어 형태로 되어있다.

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.
...,...,...
194508,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
194509,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
194510,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
194511,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


19만 4513개의 데이터로 이루어져있지만 간단히 20000개의 샘플 데이터만 가지고 기계 번역기를 구축

In [5]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:20000] # 2만개만 저장
lines.sample(10) # 랜덤으로 선택된 10개의 샘플

,src,tar
18282,I bowed politely.,Je m'inclinai poliment.
1563,We agreed.,Nous sommes tombés d'accord.
3357,I'm at home.,Je suis dans la maison.
13873,I could be next.,Je pourrais être le prochain.
10321,I fell in love.,Je suis tombée amoureuse.
4161,We're dizzy.,On a la tête qui tourne.
9267,Who disagreed?,Qui n'était pas d’accord ?
12364,We have a pool.,Nous avons une piscine.
16979,You're annoying.,Tu es chiant.
1182,I'm lucky.,Je suis chanceux.


번역 문장에 해당하는 프랑스어 데이터에는 시작을 의미하는 심볼 \<sos\>과 종료를 의미하는 심볼 \<eos\>를 넣어주어야 한다!

In [6]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
17053,You're involved.,\t Tu es impliqué. \n
15677,The moon is out.,\t La lune est visible. \n
18381,I didn't see you.,\t Je ne vous ai pas vu. \n
18763,I made it myself.,\t Je l'ai fait moi-même. \n
19668,It's your choice.,\t C'est ton choix. \n
17476,Did you love Tom?,\t Aimiez-vous Tom ? \n
10502,I like picnics.,\t J'aime bien les pique-niques. \n
12136,Tom looks dead.,\t Tom semble être mort. \n
19141,I'll study a lot.,\t J'étudierai beaucoup. \n
19122,I'll go with you.,\t Je vais y aller avec toi. \n


In [7]:
# 문자 집합 구축 ( 문자 단위... A, B, C...)
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [8]:
# 문자 집합의 크기 확인
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 75
target 문장의 char 집합 : 101


지금 상태에서 인덱스를 사용해 일부만 출력하려고 하면 에러가 나기 때문에 정렬 후 사용해준다.

In [9]:
# 정렬 전 (set형태)
#print(src_vocab[45:75])
#print(tar_vocab[45:75])

In [10]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
['U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [11]:
# 각 문자에 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'Y': 47, 'a': 48, 'b': 49, 'c': 50, 'd': 51, 'e': 52, 'f': 53, 'g': 54, 'h': 55, 'i': 56, 'j': 57, 'k': 58, 'l': 59, 'm': 60, 'n': 61, 'o': 62, 'p': 63, 'q': 64, 'r': 65, 's': 66, 't': 67, 'u': 68, 'v': 69, 'w': 70, 'x': 71, 'y': 72, 'z': 73, 'é': 74}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, ',': 11, '-': 12, '.': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 3

인덱스가 부여된 문자 집합을 이용해 훈련 데이터를 정수 인코딩

In [12]:
# 인코더에 입력될 english data
encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        # 각 char을 정수로 변환
        encoded_line.append(src_to_index[char]) # 딕셔너리의 키를 이용해 숫자로 변환
    encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 62, 10], [30, 62, 10], [30, 62, 10], [31, 56, 10], [31, 56, 10]]


In [13]:
# 디코더에 입력될 French data
decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 47, 50, 3, 4, 3, 2], [1, 3, 38, 50, 67, 52, 57, 54, 13, 3, 2], [1, 3, 27, 64, 70, 56, 54, 3, 4, 3, 2], [1, 3, 44, 50, 61, 70, 69, 3, 4, 3, 2], [1, 3, 44, 50, 61, 70, 69, 13, 3, 2]]


정상적으로 정수 인코딩이 수행되었다.  
decoder_input의 결과를 살펴보면 시작에 \<sos\>, \<eos\>(\t, \n)를 넣었기 때문에 앞과 뒤의 1개의 숫자가 모두 같은 것을 확인할 수 있다.  
이제 디코더의 예측값과 비교하기 위한 실제값이 필요한데, 실제 값에는 시작 심볼에 해당하는 \<sos\>가 있을 필요가 없다. 따라서 정수 인코딩과정에서 \<sos\>를 제거해준다.

In [14]:
decoder_target = []
for line in lines.tar:
    timestep = 0 
    encoded_line = []
    for char in line:
        # 앞의 1글자(<sos>)를 제외하고 append
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 47, 50, 3, 4, 3, 2], [3, 38, 50, 67, 52, 57, 54, 13, 3, 2], [3, 27, 64, 70, 56, 54, 3, 4, 3, 2], [3, 44, 50, 61, 70, 69, 3, 4, 3, 2], [3, 44, 50, 61, 70, 69, 13, 3, 2]]


decoder_input과 decoder_target을 비교해보면 decoder_target에서 \<sos\>에 해당하는 5개의 정수가 사라진 것을 확인할 수 있다.

다음은 패딩 작업을 수행한다. 패딩을 위해서 영어 문장과 프랑스어 문장 각각에 대해서 가장 길이가 긴 샘플의 길이를 확인한다.

In [15]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 17
target 문장의 최대 길이 : 61


In [16]:
# 영어는 영어 데이터끼리 프랑스어는 프랑스어끼리 길이에 맞춰 패딩
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [17]:
encoder_input.shape, decoder_input.shape, decoder_target.shape

((20000, 17), (20000, 61), (20000, 61))

In [18]:
# 모든 값에 대해 원-핫 인코딩 수행
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [19]:
encoder_input.shape, decoder_input.shape, decoder_target.shape

((20000, 17, 75), (20000, 61, 101), (20000, 61, 101))

## Training

**현재 시점의 디코더 셀의 입력은 이전 시점의 디코더 셀의 출력을 입력받는다고 했는데 decoder_input이 필요한 이유?** : Training 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력값으로 넣어주지 않고 이전 시점의 실제 값을 현재 시점의 디코더 셀의 입력값으로 넣어줄 것이다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 연쇄 작용으로 디코더 전체의 예측이 어려워질 수 있기 때문이다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제 값을 입력으로 주는 방법을 교사 강요(Teacher forcing)라고 한다.

In [20]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [21]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [22]:
encoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>]

인코더를 살펴보면 LSTM의 hidden state는 256으로, 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True로 설정

LSTM에서는 state_h, state_c를 리턴받는데 이는 각각 hidden state와 cell state를 의미한다.두 state는 모두 디코더로 전달된다. (Context Vector)

디코더는 인코더의 마지막 hidden state를 초기 hidden state로 사용한다. 디코더도 hidden state와 cell state를 리턴하기는 하지만 Training과정에서는 사용하지 않는다. 이후 출력층에 프랑스어 단어 집합의 크기만큼 뉴런을 배치한 후 소프트맥스 함수를 사용하여 오차를 구한다.

In [23]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# return_sequences : 시퀀스 출력 여부 
# many to many 문제 or LSTM layer를 여러개로 쌓아올릴 때 사용한다.

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달. ( 이전 결과를 저장하지 않는 모습)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [24]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, 
          epochs=5, validation_split=0.2)

Epoch 1/5
250/250 [==============================] - 14s 44ms/step - loss: 1.0769 - val_loss: 0.8695
Epoch 2/5
250/250 [==============================] - 10s 41ms/step - loss: 0.6506 - val_loss: 0.7086
Epoch 3/5
250/250 [==============================] - 10s 41ms/step - loss: 0.5464 - val_loss: 0.6218
Epoch 4/5
250/250 [==============================] - 10s 41ms/step - loss: 0.4912 - val_loss: 0.5838
Epoch 5/5
250/250 [==============================] - 10s 41ms/step - loss: 0.4511 - val_loss: 0.5458


위에서 설정한 hidden_state의 크기와 epoch 수는 train date에 과적합을 만든다. 따라서 중간부터 val_loss값이 올라가는데 이번 구현에서는 주어진 데이터의 양과 태스크의 특성 상 훈련 데이터의 정확도와 과적합 방지 두 가지를 잡기가 쉽지 않기 때문에 seq2seq 매커니즘을 구현해보고 짧은 문장과 긴 문장에 대한 성능 차이를 확인하는 것에 중점을 둔다.

컴퓨터 사양의 한계로 본 예시와는 다르게 2만개의 데이터로, epoch를 5로 낮추어 진행하였다. 

## Running

seq2seq은 훈련할 때와 동작할 때의 방식이 다르다.  
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 hidden state와 cell state를 얻음
2. 2가지의 state와 \<sos\>를 디코더로 보냄
3. 디코더가 \<eos\>가 나올때까지 반복하여 다음 문자를 예측함

In [25]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [26]:
# 인코더 정의
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [27]:
# 인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [28]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [29]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Hi.
정답 문장: Salut ! 
번역 문장: Allez ! 
-----------------------------------
입력 문장: Hello!
정답 문장: Salut ! 
번역 문장: Arrêtez ! 
-----------------------------------
입력 문장: Hop in.
정답 문장: Montez. 
번역 문장: Arrêtez ! 
-----------------------------------
입력 문장: Help me!
정답 문장: Aide-moi ! 
번역 문장: Arrêtez ! 
-----------------------------------
입력 문장: Humor Tom.
정답 문장: Mettez Tom de bonne humeur. 
번역 문장: Arrêtez de conter ! 
